In [1]:
import open3d as o3d
import numpy as np
from scipy.spatial.transform import Rotation
import copy


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


# Utilizes code

### transformation

In [2]:
def readXYZfile(filename, Separator=" "):
  data = []
  
  f = open(filename,'r') 
  line = f.readline() 
  num = 0
  while line:  #按行读入点云

     x,y,z = line.split(Separator)[:3]
     data.append([float(x),float(y),float(z)])  #X坐标
     num = num + 1
     line = f.readline()
  f.close() 
  

  return data

def get_rotationMatrix_from_euler(euler_list, pattern='XYZ', ret_np=False):
    rot = Rotation.from_euler(pattern, euler_list).as_matrix()
    if ret_np:
        rot = np.array(rot, dtype=np.float)
    return list(rot)

def get_rotationMatrix_from_quat(quat_list, ret_np=False):
    rot = Rotation.from_quat(quat_list).as_matrix()
    if ret_np:
        rot = np.array(rot, dtype=np.float)
    return list(rot)

# PointCloud Coordination Transform
def transfromCP(points, transf, ret_np=False):

    rotation = np.array(transf[:3,:3], dtype=np.float32)
    translation = np.array([transf[0][3],transf[1][3], transf[2][3]], dtype=np.float32)

    print("rotation: ", rotation)
    print("translation: ", translation)
    new_points = []
    
    for point in points:
        np.resize(point,(3,1))
        new_points.append(np.matmul(rotation, point) + translation)
    
    if ret_np:
        new_points = np.array(new_points, dtype=np.float32)
    
    return new_points

# Rotate PointCloud
def rotate_pointCloud(points, rot, ret_np=False):
    new_points = []
    
    for point in points:
        np.resize(point,(3,1))
        new_points.append(np.matmul(rot, point))
    
    if ret_np:
        new_points = np.array(new_points, dtype=np.float32)
    
    return new_points

# Transform Matrix from xyzrpy
def get_transform_matrix_xyzrpy(trans_list, euler_list, pattern='XYZ', ret_np=False):
    rot_matrix = get_rotationMatrix_from_euler(euler_list, pattern)
    ret = np.zeros((4,4), dtype=np.float32)
    for i in range(3):
        for j in range(3):
            ret[i,j] = rot_matrix[i][j]
    
    ret[0,3] = trans_list[0]
    ret[1,3] = trans_list[1]
    ret[2,3] = trans_list[2]
    ret[3,3] = 1
    return ret

# Transform Matrix from xyz abcd
def get_transform_matrix_xyzabcd(xyzabcd, ret_np=False):
    rot_matrix = get_rotationMatrix_from_quat(xyzabcd[3:])
    ret = np.zeros((4,4), dtype=np.float32)
    for i in range(3):
        for j in range(3):
            ret[i,j] = rot_matrix[i][j]
    
    ret[0,3] = xyzabcd[0]
    ret[1,3] = xyzabcd[1]
    ret[2,3] = xyzabcd[2]
    ret[3,3] = 1
    return ret
    

### visualization

In [3]:

def draw_registration_result(source, target, transformation):
    transformation = np.array(transformation, dtype=np.float32)
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    # source_temp.paint_uniform_color([1, 0.706, 0])
    # target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])
    

def draw_registration_results(sources, target, transformations):
    target_temp = copy.deepcopy(target)
    sources_temp = copy.deepcopy(sources)
    # target_temp.paint_uniform_color([1, 0.706, 0])
    
    # sources_temp = [x.paint_uniform_color([0, 0.651, 0.929]) for x in sources_temp]
    
    sources_temp = [x.transform(transformations[i]) for i,x in enumerate(sources)]
    sources_temp.append(target_temp)
    o3d.visualization.draw_geometries(sources_temp)
    
def draw_registration_results_and_ret(sources, target, transformations):
    target_temp = copy.deepcopy(target)
    sources_temp = copy.deepcopy(sources)
    # target_temp.paint_uniform_color([1, 0.706, 0])
    
    # sources_temp = [x.paint_uniform_color([0, 0.651, 0.929]) for x in sources_temp]
    
    sources_temp = [x.transform(transformations[i]) for i,x in enumerate(sources)]
    sources_temp.append(target_temp)
    o3d.visualization.draw_geometries(sources_temp)
    return sources_temp

# Iterative Closest Point

### Initial position of clouds

$ rosrun tf tf_echo /camera_color_frame /camera_color_optical_frame   
- Translation: [0.000, 0.000, 0.000]
- Rotation: in Quaternion [-0.500, 0.500, -0.500, 0.500]
            in RPY (radian) [-1.571, -0.000, -1.571]
            in RPY (degree) [-90.000, -0.000, -90.000]


$ rosrun tf tf_echo /platform /camera_color_frame                         
At time 1668856518.632
- Translation: [0.171, 0.221, 0.533]
- Rotation: in Quaternion [0.036, 0.004, -0.396, 0.917]
            in RPY (radian) [0.063, 0.036, -0.814]
            in RPY (degree) [3.606, 2.081, -46.656]


At time 1668856645.232
- Translation: [0.075, -0.064, 0.479]
- Rotation: in Quaternion [0.445, -0.063, -0.164, 0.878]
            in RPY (radian) [0.931, 0.035, -0.352]
            in RPY (degree) [53.364, 2.021, -20.172]

At time 1668856816.232
- Translation: [0.782, 0.147, 0.426]
- Rotation: in Quaternion [-0.105, -0.003, -0.744, 0.660]
            in RPY (radian) [-0.137, -0.160, -1.679]
            in RPY (degree) [-7.829, -9.191, -96.222]

At time 1668856927.732
- Translation: [0.304, 0.128, 0.821]
- Rotation: in Quaternion [0.156, 0.177, -0.427, 0.873]
            in RPY (radian) [0.135, 0.458, -0.879]
            in RPY (degree) [7.759, 26.232, -50.359]

At time 1668857029.632
- Translation: [0.429, 0.123, 0.224]
- Rotation: in Quaternion [0.881, -0.435, 0.185, -0.007]
            in RPY (radian) [-2.958, -0.326, -0.947]
            in RPY (degree) [-169.492, -18.686, -54.263]

### registration of two clouds

In [4]:
# import os
# PC_dir = "/home/msi/1008_catkin_ws/src/realsenseD455_bridge/saved_points/"
# pointClouds = ["cloud2.xyz","cloud3.xyz"]

# # demo_icp_pcds = o3d.data.DemoICPPointClouds()
# target = o3d.io.read_point_cloud(os.path.join(PC_dir, "cloud1.xyz"))
# sources = [o3d.io.read_point_cloud(os.path.join(PC_dir, x)) for x in pointClouds]
# trans_inits = [get_transform_matrix([0,0,0], [0,(i+1)*3.1416/4,0]) for i in range(len(pointClouds))]
# print(trans_inits)
# threshold = 0.02
# draw_registration_results(sources, target, trans_inits)

In [5]:
# transformation = o3d.pipelines.registration.registration_icp(
#     sources[0], target, threshold, trans_inits[0],
#     o3d.pipelines.registration.TransformationEstimationPointToPoint()).transformation

# draw_registration_result(sources[0], target, transformation)

### registration of five clouds

In [6]:
import os
PC_dir = "/home/msi/1008_catkin_ws/src/realsenseD455_bridge/saved_points/"
pointClouds = ["cloud2.ply","cloud3.ply","cloud4.ply", "cloud5.ply" ]

T0 = get_transform_matrix_xyzabcd([0,0,0,-0.500, 0.500, -0.500, 0.500])
T_target = get_transform_matrix_xyzabcd([0.171, 0.221, 0.533,0.036, 0.004, -0.396, 0.9170])

sources_xyzabcd = [
    [0.075, -0.064, 0.479,0.445, -0.063, -0.164, 0.878],
    [0.782, 0.147, 0.426,-0.105, -0.003, -0.744, 0.660],
    [0.304, 0.128, 0.821,0.156, 0.177, -0.427, 0.873],
    [0.429, 0.123, 0.224,0.881, -0.435, 0.185, -0.007]
]

T_sources = [get_transform_matrix_xyzabcd(x) for x in sources_xyzabcd]


target = o3d.io.read_point_cloud(os.path.join(PC_dir, "cloud1.ply"))
sources = [o3d.io.read_point_cloud(os.path.join(PC_dir, x)) for x in pointClouds]

target.transform(np.matmul(T_target,T0))

sources = [x.transform(np.matmul(T_sources[i],T0)) for i,x in enumerate(sources)]

trans_inits = [get_transform_matrix_xyzabcd([0,0,0,0,0,0,1]) for i in range(len(sources))]

threshold = 0.02
# draw_registration_results(sources, target, trans_inits)

In [7]:
print("Initial alignment")
print(len(sources))
for i in range(len(sources)):
    evaluation = o3d.pipelines.registration.evaluate_registration(
        sources[i], target, threshold, trans_inits[i])
    print(evaluation)

Initial alignment
4
RegistrationResult with fitness=8.847587e-01, inlier_rmse=6.282538e-03, and correspondence_set size of 87592
Access transformation to get result.
RegistrationResult with fitness=8.852450e-01, inlier_rmse=8.347609e-03, and correspondence_set size of 126814
Access transformation to get result.
RegistrationResult with fitness=9.292545e-01, inlier_rmse=6.897452e-03, and correspondence_set size of 104267
Access transformation to get result.
RegistrationResult with fitness=9.322340e-01, inlier_rmse=8.164335e-03, and correspondence_set size of 85085
Access transformation to get result.


In [8]:
print("Apply point-to-point ICP")

transformations = [o3d.pipelines.registration.registration_icp(
    source, target, threshold, trans_inits[i],
    o3d.pipelines.registration.TransformationEstimationPointToPoint()).transformation for i,source in enumerate(sources)]
trans_inits = transformations
clouds_dent = draw_registration_results_and_ret(sources, target, transformations)

dent_cloud = None
for cloud in clouds_dent:
    if(dent_cloud == None):
        dent_cloud = cloud
    else:
        dent_cloud = dent_cloud + cloud

Apply point-to-point ICP


# Down Sampling & Filtering

In [9]:
len(dent_cloud.points)
voxel_down_sample_cloud = dent_cloud.voxel_down_sample(voxel_size=0.005)
uniform_down_sample_cloud = dent_cloud.uniform_down_sample(every_k_points=3)


In [13]:
# nb_neighbors:最近k个点    std_ratio:基于标准差的阈值，越小滤除点越多
cl,ind = voxel_down_sample_cloud.remove_statistical_outlier(nb_neighbors=150,std_ratio=0.1)
k_nei_pcd = voxel_down_sample_cloud.select_by_index(ind)
o3d.visualization.draw_geometries([k_nei_pcd])

In [39]:
# nb_points:基于球体内包含点数量的阈值  radius:半径
cl,ind = voxel_down_sample_cloud.remove_radius_outlier(10,radius=0.02)
remove_radius_outlier_pcd = voxel_down_sample_cloud.select_by_index(ind)
o3d.visualization.draw_geometries([remove_radius_outlier_pcd])

# Visualization

### take photos from different position

In [25]:
import matplotlib.pyplot as plt

def custom_draw_geometry_with_camera_trajectory(pcd,
                                                camera_trajectory_path):
    custom_draw_geometry_with_camera_trajectory.index = -1
    custom_draw_geometry_with_camera_trajectory.trajectory =\
        o3d.io.read_pinhole_camera_trajectory(camera_trajectory_path)
    custom_draw_geometry_with_camera_trajectory.vis = o3d.visualization.Visualizer(
    )
    # custom_draw_geometry_with_camera_trajectory.ctr = custom_draw_geometry_with_camera_trajectory.vis.get_view_control()
    # custom_draw_geometry_with_camera_trajectory.ctr.change_field_of_view(step=90.0)
    image_path = os.path.join("/home/msi/1008_catkin_ws/src/realsenseD455_bridge/scripts/open3D_pictures", 'image')
    if not os.path.exists(image_path):
        os.makedirs(image_path)
    # depth_path = os.path.join(test_data_path, 'depth')
    # if not os.path.exists(depth_path):
    #     os.makedirs(depth_path)

    def move_forward(vis):
        # This function is called within the o3d.visualization.Visualizer::run() loop
        # The run loop calls the function, then re-render
        # So the sequence in this function is to:
        # 1. Capture frame
        # 2. index++, check ending criteria
        # 3. Set camera
        # 4. (Re-render)
        ctr = vis.get_view_control()
        ctr.rotate(18.0,0.0)
        glb = custom_draw_geometry_with_camera_trajectory
        if glb.index >= 0:
            print("Capture image {:05d}".format(glb.index))
            
            image = vis.capture_screen_float_buffer(False)
            plt.imsave(os.path.join(image_path, '{:05d}.png'.format(glb.index)),
                       np.asarray(image),
                       dpi=1)
            
            # depth = vis.capture_depth_float_buffer(False)
            # plt.imsave(os.path.join(depth_path, '{:05d}.png'.format(glb.index)),
            #            np.asarray(depth),
            #            dpi=1)
             
            # vis.capture_depth_image("depth/{:05d}.png".format(glb.index), False)
            # vis.capture_screen_image("image/{:05d}.png".format(glb.index), False)
        glb.index = glb.index + 1
        if glb.index < len(glb.trajectory.parameters):
            pass
            # ctr.convert_from_pinhole_camera_parameters(
            #     glb.trajectory.parameters[glb.index], allow_arbitrary=True)
        else:
            custom_draw_geometry_with_camera_trajectory.vis.\
                register_animation_callback(None)
        return False

    vis = custom_draw_geometry_with_camera_trajectory.vis
    vis.create_window()
    vis.add_geometry(pcd)
    ctr = vis.get_view_control()
    ctr.set_front([-0.86550403147135757, 0.50006642277345881, -0.028919618278666261])
    ctr.set_zoom(0.7)
    ctr.set_lookat([0.61088458536769674, -0.30636349079498498, 0.42844215833833982])
    ctr.set_up([-0.022685560375805946, 0.018543053043332553, 0.99957066810419548])
    vis.register_animation_callback(move_forward)
    vis.run()
    vis.destroy_window()
			
def take_photos(pcd):
    sample_data = o3d.data.DemoCustomVisualization()

    # pcd = o3d.io.read_point_cloud(sample_data.point_cloud_path)
    print("6. Customized visualization playing a camera trajectory")
    custom_draw_geometry_with_camera_trajectory(
        pcd, sample_data.camera_trajectory_path)

start pos
{
	"class_name" : "ViewTrajectory",
	"interval" : 29,
	"is_loop" : false,
	"trajectory" : 
	[
		{
			"boundingbox_max" : [ 0.93243532001533591, -0.21073464263476771, 0.77252421919861525 ],
			"boundingbox_min" : [ 0.28933385072005746, -0.40199233895520226, 0.084360097478064389 ],
			"field_of_view" : 60.0,
			"front" : [ -0.86550403147135757, 0.50006642277345881, -0.028919618278666261 ],
			"lookat" : [ 0.61088458536769674, -0.30636349079498498, 0.42844215833833982 ],
			"up" : [ -0.022685560375805946, 0.018543053043332553, 0.99957066810419548 ],
			"zoom" : 0.69999999999999996
		}
	],
	"version_major" : 1,
	"version_minor" : 0
}

In [ ]:
take_photos(remove_radius_outlier_pcd)

### show animations

In [13]:
import time
def custom_draw_geometry_with_rotation(pcd):
    def rotate_view(vis):
        ctr = vis.get_view_control()
        ctr.rotate(10.0,0.0)
        return False
    
    o3d.visualization.draw_geometries_with_animation_callback([pcd], rotate_view)


In [ ]:
class RenderClass(object):
    def __init__(self, pcd):
        self.pcd = pcd
        self.current_render_index = 0
    
    def vis_callback(self, vis):
        if self.current_render_index < len(self.sampled_index):
            mesh_sphere = o3d.geometry.TriangleMesh.create_sphere(radius=0.5)
            mesh_sphere.paint_uniform_color([0.8, 0.1, 0.2])
            mesh_sphere.translate(self.points[self.sampled_index[self.current_render_index]])
            self.current_render_index = self.current_render_index + 1
            time.sleep(0.1)
            vis.add_geometry(mesh_sphere, False)
        
        # else:
        #     vis.clear_geometries()
        #     vis.add_geometry(self.pcd, False)
        #     self.current_render_index = 0

In [31]:
# o3d.visualization.draw_geometries([voxel_down_sample_cloud])
custom_draw_geometry_with_rotation(voxel_down_sample_cloud)

In [15]:
rc = RenderClass(voxel_down_sample_cloud)
o3d.visualization.draw_geometries_with_animation_callback([voxel_down_sample_cloud], rc.vis_callback)

# Save

In [41]:
o3d.io.write_point_cloud("/home/msi/1008_catkin_ws/src/realsenseD455_bridge/saved_points/ICPResult.ply",remove_radius_outlier_pcd)

True